In [1]:
import sys
import os
import unittest
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
from osgeo import gdal

In [2]:
# add parent dir to the path
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

sys.path.append(parent_dir)


In [10]:
# importing your modules
try:
    from utility_function.interpolations_methods import (
        create_gridx_gridy,
        interpolate_pnt_idw)
    from utility_function.vector_process import (
        find_minimum_rectangle_gdf,
        distance_nearest_neighbor_pnt)
    
    from utility_function.raster_process import (
        save_raster,
        calculate_raster_extent)
    
    
    print("Modules imported successfully.")
except ImportError as e:
    print(f"Error importing modules: {e}")

Modules imported successfully.


In [11]:
class TestGeospatialFunctions(unittest.TestCase):

    def setUp(self):
        # Create a sample GeoDataFrame for testing
        self.data = {
            'geometry': [Point(1, 1), Point(1, 20), Point(20, 1), Point(20, 20)],
            'value': [10, 20, 30, 40]
        }
        self.gdf = gpd.GeoDataFrame(self.data, crs="EPSG:3857")

    def test_create_gridx_gridy(self):
        grid_x, grid_y = create_gridx_gridy(self.gdf, cell_size=1, buffer=1)
        self.assertEqual(grid_x.shape[0], grid_y.shape[0])
        self.assertEqual(grid_x.shape[1], grid_y.shape[1])

    def test_interpolate_pnt_idw(self):
        result = interpolate_pnt_idw(self.gdf, 'value', cell_size=1, distance_coef=2, buffer=1)
        self.assertIn('Image', result)
        self.assertIn('GeoT_im', result)
        self.assertIn('ProjR_im', result)

    def test_find_minimum_rectangle_gdf(self):
        bnd_gdf = find_minimum_rectangle_gdf(self.gdf, buffer=1)
        self.assertEqual(len(bnd_gdf), 1)
        self.assertIsInstance(bnd_gdf.iloc[0].geometry, Polygon)

    def test_distance_nearest_neighbor_pnt(self):
        result = distance_nearest_neighbor_pnt(self.gdf)
        self.assertEqual(len(result), self.gdf.shape[0])




class TestCalculateRasterExtent(unittest.TestCase):
    def test_calculate_raster_extent(self):
        # Test case 1
        GeoT_im = [100.0, 10.0, 0.0, 200.0, 0.0, -10.0]
        raster_size = (10, 10)
        expected_extent = [100.0, 200.0, 100.0, 200.0]
        result = calculate_raster_extent(GeoT_im, raster_size)
        self.assertEqual(result, expected_extent, "The calculated extent does not match the expected extent.")

        # Test case 2
        GeoT_im = [50.0, 5.0, 0.0, 150.0, 0.0, -5.0]
        raster_size = (20, 20)
        expected_extent = [50.0, 150.0, 50.0, 150.0]
        result = calculate_raster_extent(GeoT_im, raster_size)
        self.assertEqual(result, expected_extent, "The calculated extent does not match the expected extent.")

        # Test case 3
        GeoT_im = [0.0, 1.0, 0.0, 100.0, 0.0, -1.0]
        raster_size = (100, 100)
        expected_extent = [0.0, 100.0, 0.0, 100.0]
        result = calculate_raster_extent(GeoT_im, raster_size)
        self.assertEqual(result, expected_extent, "The calculated extent does not match the expected extent.")





class TestRasterFunction(unittest.TestCase):
    def setUp(self):
        # Create a dictionary
        self.raster_data = {
            'Image': np.random.random((5,5)),
            'GeoT_im': (0, 1, 0, 10, 0, -1),
            'band_name': ["Band 1"],
            'ProjR_im': 'EPSG:4326',
            'nodata': -9999
            }
        
        self.file_name = 'test_raster1.tif'
        
    def tearDown(self):
        # Remove the file after the test
        if os.path.exists(self.file_name):
            os.remove(self.file_name)

    def test_save_raster(self):
        save_raster(self.raster_data, self.file_name)

        # Check if file is created
        self.assertTrue(os.path.exists(self.file_name), "Output file was not created.")

        # Open the file and check contents
        dataset = gdal.Open(self.file_name)

        # Check geotransform
        geo_transform = dataset.GetGeoTransform()
        self.assertEqual(geo_transform, tuple(self.raster_data["GeoT_im"]), "GeoTransform does not match.")

        # Check band data
        band = dataset.GetRasterBand(1)
        band_data = band.ReadAsArray()
        np.testing.assert_array_almost_equal(band_data, self.raster_data["Image"], decimal=5, err_msg="Band data does not match.")

        # Check no data value
        no_data_value = band.GetNoDataValue()
        self.assertEqual(no_data_value, self.raster_data["nodata"], "NoData value does not match.")

        # Check band name
        band_name = band.GetDescription()
        self.assertEqual(band_name, self.raster_data["band_name"][0], "Band name does not match.")

    

if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_calculate_raster_extent (__main__.TestCalculateRasterExtent) ... ok
test_create_gridx_gridy (__main__.TestGeospatialFunctions) ... ok
test_distance_nearest_neighbor_pnt (__main__.TestGeospatialFunctions) ... ok
test_find_minimum_rectangle_gdf (__main__.TestGeospatialFunctions) ... ok
test_interpolate_pnt_idw (__main__.TestGeospatialFunctions) ... ok
test_save_raster (__main__.TestRasterFunction) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.138s

OK
